In [1]:
import pandas as pd
import duckdb

In [2]:
# Connect to Database
con = duckdb.connect(database='database.db', read_only=False)

In [3]:
# Mac Path
# nets_pub_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_companies_public_maris.dta')
# nets_all_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_all_maris.dta')
# compustat_all = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/compustat.dta')
# Workstation Path
nets_all_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/hq_all_maris.dta')
nets_pub_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/hq_companies_public_maris.dta')
nets_city_state = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/nets_city_state.dta')
compustat_all = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/compustat.dta')

# ['HQDuns', 'HQCompany', 'HQAdress','HQZipCode', 'HQCity', 'HQState', 'HQArea', 'HQPhone', 'Latitude', 'Longitude', 'LevelCode']

In [4]:
nets_city_state.sample(1)

,hqduns,hqcity,hqzipcode
18612267,38195997,INDIANAPOLIS,46220


In [5]:
compustat_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42979 entries, 0 to 42978
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   gvkey      42979 non-null  int32  
 1   datadate   42979 non-null  object 
 2   fyear      42808 non-null  float64
 3   indfmt     42979 non-null  object 
 4   consol     42979 non-null  object 
 5   popsrc     42979 non-null  object 
 6   datafmt    42979 non-null  object 
 7   tic        42979 non-null  object 
 8   hqcompany  42979 non-null  object 
 9   curcd      42979 non-null  object 
 10  costat     42979 non-null  object 
 11  hqaddress  42979 non-null  object 
 12  add2       42979 non-null  object 
 13  add3       42979 non-null  object 
 14  add4       42979 non-null  object 
 15  hqzipcode  42979 non-null  object 
 16  city       42979 non-null  object 
 17  conml      42979 non-null  object 
 18  state      42979 non-null  object 
 19  hqzip4     33973 non-null  float64
dtypes: flo

In [6]:
# compustat_geocoded = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/compustat_geocoded_final.dta')
compustat_geocoded = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/compustat_geocoded_final.dta')
# ['gvkey', 'hq_address_compustat', 'count', 'lat', 'lon']
print(compustat_geocoded.info())
compustat_geocoded

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42980 entries, 0 to 42979
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 42980 non-null  int32  
 1   hq_address_compustat  42980 non-null  object 
 2   count                 42980 non-null  float32
 3   lat                   41999 non-null  float64
 4   lon                   41999 non-null  float64
dtypes: float32(1), float64(2), int32(1), object(1)
memory usage: 1.6+ MB
None


,gvkey,hq_address_compustat,count,lat,lon
0,1000,,1.0,NaN,NaN
1,1001,1924 South Utica Avenue Tulsa OK,2.0,-95.968066,36.134328
2,1002,124 Industry Lane Hunt Valley MD,3.0,-76.641662,39.467797
3,1003,7700 Hall Street St. Louis MO,4.0,-90.225770,38.707338
4,1004,1100 North Wood Dale Road Wood Dale IL,5.0,-87.981054,41.986749
...,...,...,...,...,...
42975,351491,Via Puglia n. 35 Turin,42976.0,7.723420,45.106830
42976,351590,Fasanenweg 10 Leinfelden-Echterdingen,42977.0,9.145069,48.707416
42977,352262,Hamilton,42978.0,175.281099,-37.789474
42978,353444,The Heights Building 5 Weybridge,42979.0,-0.462543,51.352960


In [7]:
con.execute('DROP TABLE IF EXISTS compustat_geocoded')
con.execute('CREATE TABLE compustat_geocoded AS SELECT * FROM compustat_geocoded')
con.sql('SELECT DISTINCT COUNT(gvkey) FROM compustat_geocoded;')

┌──────────────┐
│ count(gvkey) │
│    int64     │
├──────────────┤
│        42980 │
└──────────────┘

-- START OF COMPUSTAT PREPROCESSING --

In [8]:
con.execute('DROP TABLE IF EXISTS compustat')
con.execute('CREATE TABLE compustat AS SELECT DISTINCT gvkey, hqcompany, hqzipcode, city FROM compustat_all')
con.sql('SELECT DISTINCT COUNT(*) FROM compustat;')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        42979 │
└──────────────┘

In [9]:
"""
Extract entries of compustat_geocoded that are not in all_data_cleaned
"""
query_str = """
SELECT compustat_geocoded.gvkey
FROM compustat_geocoded
LEFT JOIN compustat ON compustat_geocoded.gvkey = compustat.gvkey
WHERE compustat.gvkey IS NULL;
"""
con.sql(query_str).fetchdf()
# (duckdb.sql(query_str).to_df()).to_excel('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/preprocessed_data/missing_entries_compustat.xlsx', index=False)


,gvkey
0,9007


In [10]:
print(compustat_all.loc[compustat_all['gvkey'] == 1001])
compustat_geocoded.loc[compustat_geocoded['gvkey'] == 1001]

   gvkey    datadate   fyear indfmt consol popsrc datafmt    tic  \
1   1001  1978-12-31  1978.0   INDL      C      D     STD  AMFD.   

                 hqcompany curcd costat                hqaddress add2 add3  \
1  A & M FOOD SERVICES INC   USD      I  1924 South Utica Avenue             

  add4 hqzipcode   city                    conml state  hqzip4  
1          94104  Tulsa  A & M Food Services Inc    OK  9410.0  


,gvkey,hq_address_compustat,count,lat,lon
1,1001,1924 South Utica Avenue Tulsa OK,2.0,-95.968066,36.134328


In [11]:
"""
In this cell, we want to join the geo date from compustat_geocoded with the other data from compustat_all
Also, we want to trim and upper case the city names to make the join more robust
"""

con.execute('ALTER TABLE compustat ADD COLUMN lat DECIMAL(9,6)')
con.execute('ALTER TABLE compustat ADD COLUMN lon DECIMAL(9,6)')

# TODO: Note that lat and lon are switched in compustat_geocoded. This is probably a mistake in the geocoding process
sql_query = """
UPDATE compustat
SET lat = compustat_geocoded.lon,
    lon = compustat_geocoded.lat
FROM compustat_geocoded
WHERE compustat.gvkey = compustat_geocoded.gvkey;
"""
con.execute(sql_query)
con.sql('UPDATE compustat SET city = UPPER(TRIM(city));')

In [12]:
"""
Appearantly, there are 2196 ETFS in the compustat dataset. We want to get rid of them.
"""
sql_query = """
SELECT *
FROM compustat
WHERE REGEXP_MATCHES(hqcompany, '\\bETF\\b');
"""
con.sql(sql_query)

┌────────┬──────────────────────────────┬────────────┬───────────────┬──────────────┬──────────────┐
│ gvkey  │          hqcompany           │ hqzipcode  │     city      │     lat      │     lon      │
│ int32  │           varchar            │  varchar   │    varchar    │ decimal(9,6) │ decimal(9,6) │
├────────┼──────────────────────────────┼────────────┼───────────────┼──────────────┼──────────────┤
│  12932 │ INVESCO DORSEY WRI SM MO ETF │ 60515      │ DOWNERS GROVE │    41.822742 │   -88.031668 │
│  14435 │ ISHARES FRONTIER SLCT EM ETF │ 94105-2618 │ SAN FRANCISCO │    37.789152 │  -122.395878 │
│  16164 │ VANGUARD S&P 500 INDEX ETF   │ M5H 4E3    │ TORONTO       │    43.650511 │   -79.379722 │
│  16508 │ BMO S&P 500 INDEX ETF        │ M5K 1J5    │ TORONTO       │    43.671294 │   -79.384772 │
│  17336 │ VANECK BDC INCOME ETF        │ 10017      │ NEW YORK      │    40.751203 │   -73.974362 │
│  17589 │ MIDDLEFIELD SUS GLBL DIV ETF │ M5V 0S8    │ TORONTO       │    43.643048 │   -79

In [13]:
"""
We delete all entries that contain the substring 'ETF'
"""
sql_query = """
DELETE FROM compustat
WHERE REGEXP_MATCHES(hqcompany, '\\bETF\\b');
"""
con.execute(sql_query)
con.sql('SELECT * FROM compustat;').fetchdf()

,gvkey,hqcompany,hqzipcode,city,lat,lon
0,1005,A.B.A. INDUSTRIES INC,33782,PINELLAS PARK,27.865440,-82.703376
1,1006,ABC INDS INC,46590,WINONA LAKE,41.230724,-85.818859
2,1007,ABKCO INDUSTRIES INC,10019,NEW YORK,40.763350,-73.982401
3,1008,ABM COMPUTER SYSTEMS INC,92714,IRVINE,33.636143,-117.718777
4,1017,AEL INDUSTRIES -CL A,19446,LANSDALE,40.269510,-75.244686
...,...,...,...,...,...,...
39777,325778,MYNARIC AG,82205,GILCHING,48.091320,11.309010
39778,325867,ORPHAZYME A/S,2200,COPENHAGEN,55.698830,12.556294
39779,329579,BITFARMS LTD,M5C 1C4,TORONTO,43.649535,-79.377094
39780,331856,IMMUNIC INC,10036,NEW YORK,40.758111,-73.981505


In [14]:
"""
In this Cell we want to preprocess the names of the compustat datasets
"""
# most common substrings in company names
compustat_substring = ['INC', 'CORP', 'LTD', 'GROUP', 'CO', 'CORP']
#TODO: Maybe get rid of things like 'ENTERPRISES', 'PROPERTIES', 'INTERNATIONAL', 'TECHNOLOGIES'
#TODO: Mauybe ged rid of terms like -old or -llp
#TODO replace terms like technologie through special characters like $ or ˆ, since they are often used in company names
# other: ['etf', 'financial', 'bancorp', 'holdings', 'energy']

# query_str = """
# UPDATE compustat
#     SET conm = REGEXP_REPLACE(conm, '(\\bLLC\\b)|(\\bINC\\b)|(\\bLTD\\b)|(\\bGROUP\\b)|(\\bCO\\b)|(\\bCORP\\b)|(\\s+)|(\\-)', '', 'g')
# """
#TOOD: We replace longer terms with their abbreviations, to reduce their weight in the name, but keeping them, to improve matching score
query_str = """
UPDATE compustat
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)

In [15]:
# """
# Clean up the phone numbers from whitespaces and dashes
# """
# query_str = """
# UPDATE compustat
#     SET phone = REGEXP_REPLACE(phone, '(\\s+)|-', '', 'g')
# """
# con.execute(query_str)
# con.sql('SELECT * FROM compustat WHERE hqcompany LIKE \'%!%\' LIMIT 10').fetchdf()


-- END OF COMPUSTAT PREPROCESSING --


-- START OF NETS PREPROCESSING --


In [16]:
con.execute('DROP TABLE IF EXISTS nets_all')
con.execute('CREATE TABLE nets_all AS SELECT hqduns, hqcompany, latitude, longitude, FROM nets_all_data')
#con.sql('SELECT * FROM nets;').fetchdf().info()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
"""
In this Cell we want to preprocess the names of the NETS datasets

NETS_substring = ['LLC', 'INC', 'OF', 'GROUP', 'SERVICES', 'CO', 'CORP']
other: ['a', 'construction', 'enterprises']
"""
#TODO: get rid of things like 'ENTERPRISES', 'PROPERTIES', 'INTERNATIONAL', 'TECHNOLOGIES', 'INCORPORATED', 'CORPORATION', 'COMPANY since acronyms like inc. arent that common in nets.
#TODO: Maybe we need more preprocessing for NETS since there are a lot of entries that are not in compustat

# query_str = """
# UPDATE nets
#     SET hqcompany = REGEXP_REPLACE(hqcompany, '(\\bLLC\\b)|(\\bINC\\b)|(\\bLTD\\b)|(\\bGROUP\\b)|(\\bCO\\b)|(\\bCORP\\b)|(\\s+)|(\\-)', '', 'g')
# """
query_str = """
UPDATE nets_all
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
# """
# In this Cell we want to extract the area code and the phone number from the NETS data
# and concatenate them to one column which then will be added to the NETS data.
# Afterwards we will compare the phone numbers of the two datasets and see if they match.
# """
# con.execute('ALTER TABLE nets ADD COLUMN hqphone_concatenated VARCHAR(255)')
# query_str = """
# UPDATE nets
# SET hqphone_concatenated = REGEXP_REPLACE(
#             REGEXP_REPLACE((CONCAT(hqarea, hqphone)), '(\\s+)|-', '', 'g')
#           , '^$', '-1');
# """
# con.execute(query_str)

In [19]:
"""
In this Cell we will fix the longitudes of the NETS data
"""
#TODO: The coordinates are not correct in our NETS sample.  Google maps gives us -119.xxxx
#TODO: Lat-long coordinates for cities in United States are in range: Latitude from 19.50139 to 64.85694 and longitude from -161.75583 to -68.01197. (Source: https://latitudelongitude.org/us/)
# TODO maybe we need to adjust the coordinates according to levelcode
sql_query = """
    UPDATE nets_all
    SET longitude = -longitude;
"""
con.execute(sql_query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [21]:
"""
In this cell we will join the zip codes and cities to the NETS data
"""
con.execute('ALTER TABLE nets_all ADD COLUMN hqzipcode VARCHAR(255)')
con.execute('ALTER TABLE nets_all ADD COLUMN hqcity VARCHAR(255)')
query_str = """
UPDATE nets_all
SET hqzipcode = nets_city_state.hqzipcode,
    hqcity = nets_city_state.hqcity
FROM nets_city_state
WHERE nets_all.hqduns = nets_city_state.hqduns;
"""
con.execute(query_str)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [22]:
con.execute('SELECT * FROM nets_all LIMIT 1000').fetchdf().sample(10)

,hqduns,hqcompany,latitude,longitude,hqzipcode,hqcity
709,1006738,AEDYNINC,45.208199,-123.212898,97128,MCMINNVILLE
695,1006472,PENINSULAHEALTHALLIANCEINC,38.698002,-75.396500,19947,GEORGETOWN
416,1003586,CROOKEDCREEKRVPARK,60.320499,-151.284103,99610,KASILOF
125,1001051,SPIRITUALSCENTS,40.896999,-72.883797,11961,RIDGE
566,1005032,SOLUTEKCORP,42.324699,-71.070297,2119,BOSTON
652,1005891,PIONEERMANUFACTURINGCORP,42.370300,-71.154602,2472,WATERTOWN
664,1006006,AMERICANKOSHERPRODUCTSCO,42.283699,-71.091202,2126,MATTAPAN
26,1000231,BELLAIREBAPTISTCHURCH,29.932501,-93.932098,77642,PORT ARTHUR
990,1009455,NUSTYLECOINC,42.080601,-71.422997,2038,FRANKLIN
65,1000558,ONTIMEFREIGHT,41.294102,-72.938797,6519,NEW HAVEN


-- NETS PUB PREPROCESSING --

In [23]:
con.execute('DROP TABLE IF EXISTS nets_pub')
con.execute('CREATE TABLE nets_pub AS SELECT hqduns, hqcompany, latitude, longitude, FROM nets_pub_data')

In [24]:
"""
In this Cell we want to preprocess the names of the NETS datasets
"""
query_str = """
UPDATE nets_pub
SET hqcompany = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE( 
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                REGEXP_REPLACE(
                                                                    REGEXP_REPLACE(
                                                                        REGEXP_REPLACE(
                                                                            REGEXP_REPLACE(hqcompany, 'INCORPORATED', 'INC'),
                                                                        'LIMITED', '`LTD'),
                                                                    'CORPORATION', 'CORP'),
                                                                'COMPANY', 'CO'),
                                                            'HOLDINGS', 'HLDGS'),
                                                        'GROUP', 'GRP'),
                                                    'INTERNATIONAL', 'INTL'),
                                                'TECHNOLOGIES', '@'),
                                            'THERAPEUTICS', '^'),
                                        'PHARMACEUTICALS', '$'),
                                    'INDUSTRIES', 'IND'),
                                'TECHNOLOGY', '!'),
                            'FINANCIAL', ')'),
                        'COMMUNICATIONS', '('),
                    'PROPERTIES', '\'),
                '\\b-\\b', ''),
            '\\s+', '', 'g'),
        'BANCORPINC', '}'),
    'HLDGSINC', '{')
    )
"""
con.execute(query_str)

In [25]:
"""
In this Cell we will fix the longitudes of the NETS data
"""
#TODO: The coordinates are not correct in our NETS sample.  Google maps gives us -119.xxxx
#TODO: Lat-long coordinates for cities in United States are in range: Latitude from 19.50139 to 64.85694 and longitude from -161.75583 to -68.01197. (Source: https://latitudelongitude.org/us/)
# TODO maybe we need to adjust the coordinates according to levelcode
sql_query = """
    UPDATE nets_pub
    SET longitude = -longitude;
"""
con.execute(sql_query)

In [26]:
# con.execute('SELECT * FROM nets').fetch_df().to_excel('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/preprocessed_data/nets_preprocessed.xlsx', index=False)
# con.execute('SELECT * FROM compustat').fetchdf().to_excel('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/preprocessed_data/compustat_preprocessed.xlsx', index=False)

In [27]:
con.commit()
con.close()